In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

### For all questions below, you may use any functions from the assignment notebook.

#### 1) Generate a vector $x$ that takes on uniformly random values in the interval $[-4, 4]$. Generate a vector $y$ that is any quadratic function of $x$ (you choose!) and add normally distributed noise onto $y$. Assume $\epsilon \sim N(0, 4a)$ where $a$ is the coefficient of the linear component $x$ of the quadratic function. Plot your results.

In [ ]:
# SOLUTION
x = np.random.uniform(-4, 4, 100)
a = 1
y = 1 + a * x + x**2 + np.random.normal(0, 4 * a, 100)
plt.plot(x, y, 'o')

#### 2) Using the generated data in the previous part, find the least squares solution with the hypothesis that our data $y$ is generated from some cubic function $y = w_0 + w_1 x + w_2 x^2 + w_3 x^3$. Recall that the least squares solution when we have a feature matrix $X$ and output data $y$ is $\hat{w} = (X^T X) ^{-1} X^T y$. Report the coefficients and the MSE. What do you notice about the coefficients, especially the coefficient for $x^3$?

In [ ]:
# SOLUTION
def featurize_d(x, d):
    n = len(x)
    X = np.empty([n, d + 1])
    for de in range(d + 1):
        X_curr_deg = np.array(x ** de)
        X[:, de] = X_curr_deg
    return X

def LstSq(X, y):
    return np.linalg.inv(X.T @ X) @ X.T @ y

def MSE(y, y_pred):
    return np.mean((y-y_pred)**2)

X_deg3 = featurize_d(x, 3)
w_deg3 = LstSq(X_deg3, y)
y_pred_deg3 = X_deg3 @ w_deg3
w_deg3, MSE(y, y_pred_deg3)
# notice that the last coefficient (for x^3) is very close to 0 -- indicating it may not be very relevant to the model

#### 3) Repeat the above exercise with a featurization for $x^4$, keeping all previous polynomial features. Does the MSE increase or decrease? Looking at how the true data was generated, why is this potentially deceiving?

In [ ]:
# SOLUTION
X_deg4 = featurize_d(x, 4)
w_deg4 = LstSq(X_deg4, y)
y_pred_deg4 = X_deg4 @ w_deg4
w_deg4, MSE(y, y_pred_deg4)

#### 4) Find $A^{100} x$, where $A$ and $x$ are fixed and provided for you. The catch: you are not allowed to compute the matrix power by doing $A^{100} = A*A*A ... *A$. Below the code cell, explain your alternate solution and then print the value. Verify that your solution matches the actual value $A^{100} x$ (your kernel won't crash, don't worry). You may use any NumPy functions. You can verify your solution using NumPy's built in $\verb|np.linalg.matrix_power|$.

In [ ]:
# SOLUTION
A = np.array([2, 0, 0, 1, 2, 1, -1, 0, 1]).reshape(3,3)
x = np.array([0, -np.sqrt(2)/2, np.sqrt(2)/2])
l, v = np.linalg.eig(A)
(1 ** 100) * x, np.linalg.matrix_power(A, 100) @ x == (1 ** 100) * x

#### 5) Use $\verb|np.einsum|$ to compute $AB$ for the A and B provided below.

In [ ]:
# SOLUTION
A = np.array([2,3,5,4,2,-3]).reshape(3,2)
B = np.array(range(1, 20, 2)).reshape(2, 5)
np.einsum('ij, jk -> ik', A, B)

#### 6) Use $\verb|np.einsum|$ to define a new function $\verb|LstSqLong(X, y)|$ that takes in the same arguments as  $\verb|LstSq|$ but only uses $\verb|np.einsum|$. You may still use the function $\verb|np.linalg.inv|$. Then run the code provided to ensure your function works with basic linear data.

In [ ]:
# SOLUTION
def LstSqLong(X,y):
    assert X.shape[0] == len(y)
    y = y.reshape(len(y), 1)
    # compute X transpose
    X_t = np.einsum('ji', X)
    # compute (X^T X) ^ -1
    arg1 = np.linalg.inv(np.einsum('ij, jk -> ik', X_t, X))
    # compute # X^T y
    arg2 = np.einsum('ij, jk -> ik', X_t, y)
    # return the matrix product of arg1 and arg2
    return np.einsum('ij, jk -> ik', arg1, arg2)

x = np.linspace(-2, 2, 100)
y = 1 + 5*x + np.random.normal(0, 2, 100)
X = featurize_d(x, 1)
y_preds = X @ LstSqLong(X, y)

plt.scatter(x,y, c='b', marker='x', label='actual')
plt.scatter(x, y_preds, c='r', marker='s', label='predicted')
plt.legend(loc='upper left')
plt.show()

#### 7) Compute $A^{5}$ for the matrix $A$ given below. As before, you are not allowed to do this with $A^{5} = A*A*A*A*A$. You may need the function $\verb|np.fill_diagonal|$. Then verify your result using $\verb|np.linalg.matrix_power|$.

In [ ]:
# SOLUTION
A = np.array([6, -1, 2, 3]).reshape(2,2)
l, P = np.linalg.eig(A)
D = np.zeros((2,2))
np.fill_diagonal(D, l)
P @ D**5 @ np.linalg.inv(P), P @ D**5 @ np.linalg.inv(P) == np.linalg.matrix_power(A, 5)

#### 8) As you commute to work, you count how many green and red lights you hit. You also record the total time it takes to commute to work, and repeat this for two days. The data is provided below (there are 25 signals, you take the same route). Formulate the problem as a least squares question and solve for the relevant coefficients. Please state the meaning of the coefficients in the context of this problem, and print the coefficients. Additionally, in a real world setting, what would be the random noise in this problem?

In [ ]:
# SOLUTION

green = np.array([20, 2])
red = np.array([5, 23])
commute_times = np.array([20, 43])
X = np.vstack((green, red))
w_LS = LstSq(X, commute_times)
w_LS

#### 9) Invert the following matrix using NumPy.

In [ ]:
# SOLUTION
A = np.array([1, 0, 0, 3, 0, 5, 2, 4, 9]).reshape(3,3)
np.linalg.inv(A)

#### 10) Using the vector $x$ provided below, create 10 featurized matrices for each $d$ from $d = 1$ to $d = 10$. Then calculate the least squares coefficients, predicted values, and MSE for each featurization. Record the MSEs in the list $\verb|MSEs|$. Plot the MSE for all 10 featurizations and note down your observations.

In [ ]:
# SOLUTION
x = np.linspace(-5, 5, 100)
y = 0.2 * x**2 + 0.3*x + 0.2 + np.random.normal(0, 0.1, 100)
MSEs = []
degrees = range(1, 11)
for degree in degrees:
    X_deg = featurize_d(x, degree)
    w_deg = LstSq(X_deg, y)
    y_pred_deg = X_deg @ w_deg
    MSE = np.mean((y - y_pred_deg)**2)
    MSEs += [MSE]

plt.plot(degrees, MSEs, 'o')

#### 11) Which of the following functions can be used to multiply matrices in Python?

a) @ <br>
b) np.multiply <br>
c) np.einsum <br>
d) np.prod <br>
e) a, b, c

In [1]:
#SOLUTION: e